<a href="https://colab.research.google.com/github/DivyaShreeK-dev/sdc/blob/main/spam_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.3 MB/s eta 0:00:00


In [ ]:
# Install dependencies (if not installed)
!pip install tensorflow gradio pandas scikit-learn

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
import gradio as gr
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load SMS Spam dataset
url = "https://raw.githubusercontent.com/justmarkham/pycon-2016-tutorial/master/data/sms.tsv"
df = pd.read_csv(url, sep='\t', names=["label", "message"])

# Convert labels to numeric (spam=1, ham=0)
df['label'] = LabelEncoder().fit_transform(df['label'])

# Tokenize and prepare input text
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(df["message"])
sequences = tokenizer.texts_to_sequences(df["message"])
padded_sequences = pad_sequences(sequences, maxlen=50)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, df["label"], test_size=0.2, random_state=42)

# Build ANN Model
model = Sequential([
    Embedding(input_dim=5000, output_dim=16, input_length=50),
    GlobalAveragePooling1D(),
    Dense(16, activation="relu"),
    Dense(1, activation="sigmoid")  # Binary classification (spam/ham)
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), verbose=2)

# Prediction function for Gradio
def predict_spam(text):
    sequence = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(sequence, maxlen=50)
    prediction = model.predict(padded)[0][0]
    return {"Spam": float(prediction), "Not Spam": float(1 - prediction)}

# Create Gradio UI
interface = gr.Interface(fn=predict_spam, inputs=gr.Textbox(lines=2, placeholder="Enter a message..."), outputs=gr.Label())

# Launch Gradio App
interface.launch(share=True)


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


140/140 - 3s - 19ms/step - accuracy: 0.8582 - loss: 0.3904 - val_accuracy: 0.8664 - val_loss: 0.3280
Epoch 2/10
140/140 - 1s - 4ms/step - accuracy: 0.8658 - loss: 0.3144 - val_accuracy: 0.8664 - val_loss: 0.2972
Epoch 3/10
140/140 - 1s - 4ms/step - accuracy: 0.8667 - loss: 0.2671 - val_accuracy: 0.8700 - val_loss: 0.2340
Epoch 4/10
140/140 - 1s - 5ms/step - accuracy: 0.9233 - loss: 0.1852 - val_accuracy: 0.9605 - val_loss: 0.1399
Epoch 5/10
140/140 - 1s - 4ms/step - accuracy: 0.9726 - loss: 0.1065 - val_accuracy: 0.9785 - val_loss: 0.0872
Epoch 6/10
140/140 - 0s - 3ms/step - accuracy: 0.9791 - loss: 0.0732 - val_accuracy: 0.9839 - val_loss: 0.0658
Epoch 7/10
140/140 - 1s - 4ms/step - accuracy: 0.9834 - loss: 0.0558 - val_accuracy: 0.9874 - val_loss: 0.0534
Epoch 8/10
140/140 - 0s - 3ms/step - accuracy: 0.9865 - loss: 0.0463 - val_accuracy: 0.9874 - val_loss: 0.0469
Epoch 9/10
140/140 - 0s - 3ms/step - accuracy: 0.9877 - loss: 0.0395 - val_accuracy: 0.9892 - val_loss: 0.0439
Epoch 10/10